In [1]:
from splinter import Browser
import time
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import datetime
import warnings
warnings.filterwarnings("ignore")

starttime = datetime.datetime.now()

In [2]:
#cd Desktop/"Car Project"
#curl -LJO https://raw.githubusercontent.com/BShim92/CarScraper/master/CarProject.ipynb


In [3]:
### ONLY CHANGE THESE ###

#zip code for KBB
zipcode = 94024

#how long to pause at certain times in the script
waittime = 1

#how many craigslist pages to search (120 per page)
pagestosearch = 3

#craigslist url with filters (without any is fine too) and the first page of the search
clurl = 'https://sfbay.craigslist.org/search/cto?auto_title_status=1&auto_title_status=3&auto_title_status=4&auto_title_status=5&auto_title_status=6'

In [4]:
pagelist = []

for pagenumber in range(pagestosearch):

    clpage = clurl + '&s=' + str(pagenumber * 120)
    
    clpage = requests.get(clpage)
    clpage = bs(clpage.text, 'html.parser')
    clpage = clpage.find('ul', class_='rows')


    for page in clpage:
        try:
            pagelist.append(page.find('a').attrs['href'])
        except:
            pass

    pagelist = list(set(pagelist))

len(pagelist)

240

In [5]:
# pagelist = pagelist[:25]

In [6]:
cldf = pd.DataFrame()

for page in pagelist:
    
    try:
        attrdict = {}

        attrdict['url'] = page

        page = requests.get(page).text
        page = bs(page, 'html.parser')

        attrgroups = page.find_all('p', class_ ='attrgroup')

        attrdict['clprice'] = page.find('span', class_='price').text.replace('$', '')

        car = attrgroups[0]
        car = car.find('span').text
        attrdict['fullcar'] = car
        attrdict['make'] = car.split(' ', 1)[-1].split(' ', 1)[0]
        attrdict['model'] = car.split(' ', 1)[-1].split(' ', 1)[-1]
        attrdict['year'] = car.split(' ')[0]


        attrs = attrgroups[1].find_all('span')

        for attr in attrs:
            try:
                attr = attr.text
                specname = attr.split(':')[0]
                specvalue = attr.split(': ')[1]
                attrdict[specname] = specvalue
            except:
                continue

        cldf = cldf.append(attrdict, ignore_index = True)
    except Exception as e:
        print(e, attrdict['url'])

'NoneType' object has no attribute 'text' https://sfbay.craigslist.org/eby/cto/d/pleasanton-1939-ford-truck-flathead/6989398102.html


In [10]:
# cldf = cldfextra


In [11]:
cldfextra = cldf
list(cldf.columns)
cldf = cldf[['make', 'model', 'year', 'clprice',  'odometer', 'paint color', 'VIN', 'fullcar', 'url' ]]



In [12]:
nonulls = ['make', 'model', 'year', 'clprice',  'odometer', 'paint color']

for column in nonulls:
    cldf = cldf[cldf[column].notnull()]
    
cldf = cldf.drop_duplicates(keep='first', subset=['VIN', 'fullcar'])


cldf = cldf.reset_index(drop=True)
print(cldf.shape)
cldf.head()

(152, 9)


make      model  year clprice odometer paint color                VIN  \
0       gmc  box truck  2003    6500   266000       white                NaN   
1     Honda      pimot  2011    6800   168000      silver                NaN   
2    toyota   camry CE  2000    2500   156000      custom  4T1BG22KXYU943284   
3    Toyota    4Runner  2016   27500    56500       white  JTEBU5JR0G5320175   
4  jeepster   commando  1969    8900    52000        blue       870501750686   

                  fullcar                                                url  
0      2003 gmc box truck  https://sfbay.craigslist.org/pen/cto/d/brisban...  
1        2011 Honda pimot  https://sfbay.craigslist.org/pen/cto/d/daly-ci...  
2    2000 toyota camry CE  https://sfbay.craigslist.org/scz/cto/d/aptos-2...  
3     2016 Toyota 4Runner  https://sfbay.craigslist.org/eby/cto/d/oakland...  
4  1969 jeepster commando  https://sfbay.craigslist.org/nby/cto/d/napa-19...

In [27]:
# cldf = cldf[:2]

In [29]:
# if make.lower() in ['fiat', 'infiniti']:
#     make = make.upper()
# else:
#     make = make.capitalize()

# if make.lower() == 'vw':
#     make = 'Volkswagen'

# if color == 'Grey':
#                 color = 'Gray'

makedict = {
    'vw': 'Volkswagen',
    'fiat': 'FIAT',
    'infiniti': 'INFINITI',
    'mercedes': 'Mercedes-Benz',
    'gmc': 'GMC',
    'bmw': 'BMW',
    'mercedes-benz': 'Mercedes-Benz'
}

for i, x in cldf.iterrows():
    cldf.iat[i, cldf.columns.get_loc("make")] = x['make'].lower()
        
for i, x in cldf.iterrows():
    if x['make'].lower() in list(makedict.keys()):
        cldf.iat[i, cldf.columns.get_loc("make")] = makedict[x['make'].lower()]
        
colordict = {
    'grey': 'gray',
}
        
for i, x in cldf.iterrows():
    if x['paint color'].lower() in list(colordict.keys()):
        cldf.iat[i, cldf.columns.get_loc("paint color")] = colordict[x['paint color'].lower()]

cldf


    

make                     model  year clprice odometer  \
0   Mercedes-Benz             Mercedes-Benz  2010   10000    92000   
1           acura               mdx touring  2003    3600   252000   
2            ford  f150 4wd harley davidson  2000    6900   155000   
3       chevrolet                Malibu LTZ  2009    4300   101000   
4          toyota                Highlander  2004    4900   185000   
5            FIAT  500x Trekking Plus (FWD)  2016   14996    35000   
6            ibmw          328i convertible  1998    5000      182   
7            1999           Toyota Camry LE  1999    1600   200000   
8            ford                escape xlt  2008    4500   148000   
9   Mercedes-Benz                     ML350  2009    9999   135000   
10          honda                     civic  2003     300   225000   
11         toyota       4Runner SR5 Limited  2010   19500    94200   
12           ford                 focus zx5  2004    4200   191000   
13           ford      f150 crew cab lariat  2005    5950   196000   
14         nissan                    Altima  2008    2700   170000   
15           ford                 rdstr p/u  1929   24500     2800   
16            BMW                        M3  2003   19500    71500   
17         subaru      outback 2.5i limited  2013   13200    89000   
18         toyota        Sienna XLE Limited  2006    9800       85   
19          lexus                    GS 350  2013   23000    90000   
20           audi                        r8  2009   58975    58000   
21           ford                  flex sel  2018     292    21178   
22         toyota                    tundra  2014   29800    74000   
23        hyundai                    sonata  2011    4300   151998   
24          honda                     Civic  2018   14995    11000   
25     volkswagen           crew cab pickup  1968    8500   100000   
26           land         rover defender 90  1994   69000   100000   
27            BMW                      335i  2011   11000   104888   
28          volvo                       s70  1998    1750   119000   
29       cadillac                       srx  2009   15000    75000   
..            ...                       ...   ...     ...      ...   
36  Mercedes-Benz           Benz  SLK32 AMG  2003   10000      147   
37         nissan               Frontier SE  2010   12750    74500   
38          chevy                        sl  1994    1200   263000   
39        lincoln       continental mark iv  1973    8500   129000   
40            BMW                      535d  2014   14500   139000   
41  Mercedes-Benz                      e350  2009    9000   136000   
42         toyota                      RAV4  2001    4000   173000   
43            BMW                328i Wagon  2016   32000    39000   
44     mitsubishi          montero sport ls  2002    2999   114000   
45         nissan                    altima  1995    1850   123000   
46          honda           Accord Ex Coupe  2002    2400   155400   
47      chevrolet          silverado 3500hd  2004   10000   207000   
48  Mercedes-Benz             benz e63s amg  2018   86000    34200   
49         toyota                   corolla  2010    7900   107500   
50     volkswagen                     jetta  1999    2500    89525   
51          mazda                   protege  2000    1850   170000   
52         subaru               Forester XT  2004    4700   174000   
53          tesla                   Model 3  2017   46500    31800   
54          honda                       CRV  2005    5900   135000   
55            BMW                 Alpina B7  2007    9900   144000   
56          honda                   element  2006    6750   132227   
57           audi           S6 Premium Plus  2016   44990    14000   
58  Mercedes-Benz                 benz e350  2006    3250   140000   
59          honda                  civic lx  2008    7000    73548   
60      chevrolet                   classic  2004    1000   136725   
61         jaguar      

In [30]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


In [ ]:
cldf['kbbprice'] = ''
cldf['kbburl'] = ''

for j, y in cldf.iterrows():

    print('----------------------')
    
    for x in range(1):
        try:
            kbbprice = 0

            year = y['year']
            make = y['make']
            model = y['model'].upper()
            mileage = y['odometer']
            color = y['paint color'].capitalize()



            browser.visit('https://www.kbb.com/whats-my-car-worth/')

            browser.find_by_id('yearDropdown').click()
            browser.find_option_by_text(year).first.click()
            browser.find_by_id('yearDropdown').click()

            time.sleep(1 * x + waittime)

            browser.find_by_id('makeDropdown').click()
            try:
                browser.find_option_by_text(make).first.click()
            except:
                try:
                    browser.find_option_by_text(make.capitalize()).first.click()
                except:
                    browser.find_option_by_text(make.upper()).first.click()
                
            browser.find_by_id('makeDropdown').click()

            time.sleep(1 * x + waittime)

            browser.find_by_id('modelDropdown').click()
            
            try:
                browser.find_option_by_text(model).first.click()
            except:
                try:
                    modelwords = re.split(' |-', model)

                    models = bs(browser.html, 'html.parser')
                    models = models.find(id='modelDropdown')
                    models = models.find_all('option')

                    modelcountdict = {}

                    for modeloptions in models:
                        modelcountdict[modeloptions.text] = 0
                        for word in modelwords:
                            if word.lower() in modeloptions.text.lower():
                                modelcountdict[modeloptions.text] += 1
                    chosenmodel = max(modelcountdict, key=modelcountdict.get)
                    
                    browser.find_option_by_text(chosenmodel).first.click()
                    
                except:
                    break
                    
            browser.find_by_id('modelDropdown').click()

            time.sleep(1 * x + waittime)

            browser.fill('mileage', mileage)

            time.sleep(1 * x + waittime)

            browser.fill('zipcode', zipcode)

            time.sleep(1 * x + waittime)

            browser.find_by_id('submitButton').click()

            time.sleep(1 * x + waittime)

            try:
                browser.find_by_text('No thanks').click()
            except:
                pass

            if browser.url == 'https://www.kbb.com/whats-my-car-worth/':
                break
                
                

            if 'styles' in browser.url:
                
                try:
                    browser.find_by_text('No thanks').click()
                except:
                    pass
                
                try:
                    browser.execute_script('document.getElementById("sedanGroup").click();')
                except:
                    pass
                
                modelwords = re.split(' |-', model)
                modelwords

                styles = bs(browser.html, 'html.parser')
                styles = styles.find_all('div', class_='style-name-container')

                stylecountdict = {}

                for style in styles:
                    stylecountdict[style.text] = 0
                    for word in modelwords:
                        if word.lower() in style.text.lower():
                            stylecountdict[style.text] += 1

                chosenstyle = max(stylecountdict, key=stylecountdict.get)
                chosenstyle = chosenstyle.split('\n')
                chosenstyle = chosenstyle[1]
                chosenstyle

                browser.execute_script("document.getElementsByClassName('js-style-option')[0].click()")

                browser.execute_script("for (i = 0; i < document.getElementsByClassName('style-radio').length; i++){\
                if(document.getElementsByClassName('style-name')[i].innerText == '" + chosenstyle +"') {\
                document.getElementsByClassName('style-radio')[i].click()\
                    }\
                }")

                time.sleep(2 * (x+1) + waittime)

                browser.find_by_id('stylesNextButton').click()

                if 'styles' in browser.url:
                    break

            browser.execute_script("document.getElementById('standardRadio').click()")

            time.sleep(1 + waittime)

            browser.execute_script("for (i = 0; i < document.getElementsByClassName('option-input').length; i++){\
                        if(document.getElementsByClassName('option-input')[i].innerText.trim() == '" + color + "') {\
                        document.getElementsByClassName('option-input')[i].getElementsByTagName('input')[0].click()\
        }\
    }")

            time.sleep(1 + waittime)

            browser.execute_script("for (i = 0; i < document.getElementsByClassName('title').length; i++){\
                        if(document.getElementsByClassName('title')[i].innerText == 'Good') {\
                        document.getElementsByClassName('title')[i].click()\
        }\
    }")
            
            time.sleep(1 + waittime)

            browser.find_by_id('stylesNextButton').click()

            if 'pricetype' not in browser.url:
                break

            try:
                browser.find_by_text('No thanks').click()
            except:
                pass

            for getprice in range(60):
                try:
#                     browser.execute_script("document.getElementsByClassName('js-private-party-tab')[0].click()")

                    kbbprice = browser.html
                    kbbprice = kbbprice.split('data-privateparty-price="')[-1]
                    kbbprice = kbbprice.split('"')[0]
                    kbbprice = float(kbbprice)
                    
                    print(j, '.  ', kbbprice)
                    
                    cldf.iat[j, cldf.columns.get_loc("kbbprice")] = float(kbbprice)
                    tradeinurl = browser.url
                    cldf.iat[j, cldf.columns.get_loc("kbburl")] = browser.url.replace('pricetype=trade-in', 'pricetype=private-party')
                    break
                except:
                    time.sleep(.5)
     
            
            break
        except Exception as e:
            print(e)
            print(f'year = {year}')
            print(f'make = {make}')
            print(f'model = {model}')
            print(f'mileage = {mileage}')
            print(f'color = {color}')
            print(j, browser.url)
            time.sleep(1 * x + waittime)

----------------------
Message: invalid element state: Element is not currently interactable and may not be manipulated
  (Session info: chrome=76.0.3809.100)

year = 2010
make = Mercedes-Benz
model = MERCEDES-BENZ
mileage = 92000
color = Gray
0 https://www.kbb.com/whats-my-car-worth/
----------------------
1 .   2897.0
----------------------
2 .   2114.0
----------------------
3 .   4334.0
----------------------
4 .   4721.0
----------------------
5 .   11631.0
----------------------
no elements could be found with option by text "IBMW"
year = 1998
make = ibmw
model = 328I CONVERTIBLE
mileage = 182
color = Silver
6 https://www.kbb.com/whats-my-car-worth/
----------------------
Message: element click intercepted: Element <select id="modelDropdown" name="modelname" class="dropdown-one" data-default-option="Model" disabled="" required="">...</select> is not clickable at point (161, 473). Other element would receive the click: <div class="form-group">...</div>
  (Session info: chrome=76.0

In [13]:
cldf.replace('', 0, inplace=True)

cldf = cldf[cldf['kbbprice'] != 0]

cldf['clprice'] = cldf['clprice'].astype(float)
cldf['kbbprice'] = cldf['kbbprice'].astype(float)
cldf['profit'] = cldf['kbbprice'] - cldf['clprice']
cldf['profit/clprice'] = cldf['profit']/cldf['clprice']
cldf = cldf.sort_values(by='profit/clprice', ascending=False)
cldf

try:
    browser.quit()
except:
    pass

In [14]:
cldf.to_csv('Cars.csv')

In [15]:
print(datetime.datetime.now()-starttime)

0:02:18.040066


In [16]:
# browser.execute_script(f'document.getElementById("yearDropdown").value = "{year}";')
# browser.find_by_xpath('//option[@value="2015"]').click()
# browser.find_by_xpath('//option[@value="2015"]').click()
# browser.find_by_xpath('//option[@value="2015"]').click()